## Data preprocessing

In [1]:
import scipy
import librosa
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
import scipy.io.wavfile
import time
import IPython
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import json
from PIL import Image
from torchvision import datasets, transforms

In [ ]:
!cp /content/drive/MyDrive/rgb.zip /content/rgb.zip
!cp /content/drive/MyDrive/depth.zip /content/depth.zip
!cp /content/drive/MyDrive/labels.zip /content/labels.zip

In [ ]:
!unzip /content/drive/MyDrive/rgb.zip
!unzip /content/drive/MyDrive/depth.zip
!unzip /content/drive/MyDrive/labels.zip

## Data preprocessing 2

In [1]:
import scipy
import librosa
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
import scipy.io.wavfile
import time
import IPython
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import json
from PIL import Image
from torchvision import datasets, transforms

In [3]:
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD/crops_rgb_train.zip /content/crops_rgb_train.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD/crops_rgb_test.zip /content/crops_rgb_test.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD/crops_depth_train.zip /content/crops_depth_train.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD/crops_depth_test.zip /content/crops_depth_test.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD/labels_test.zip /content/labels_test.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD/labels_train.zip /content/labels_train.zip


In [ ]:
os.makedirs('/content/train',exist_ok=True)
os.makedirs('/content/test',exist_ok=True)
!unzip /content/crops_rgb_train.zip -d /content/train/
!unzip /content/crops_rgb_test.zip -d /content/test/
!unzip /content/crops_depth_train.zip -d /content/train/
!unzip /content/crops_depth_test.zip -d /content/test/
!unzip /content/labels_train.zip -d /content/train/
!unzip /content/labels_test.zip -d /content/test

## Efficient

In [48]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""

import torch
import torch.nn as nn
import math

__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=1, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(4, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def extract(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        feature = x.view(x.size(0), -1)
        x = self.classifier(feature)

        return feature, x


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

## Mobile

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init



class hswish(nn.Module):
    def forward(self, x):
        out = x * F.relu6(x + 3, inplace=True) / 6
        return out


class hsigmoid(nn.Module):
    def forward(self, x):
        out = F.relu6(x + 3, inplace=True) / 6
        return out


class SeModule(nn.Module):
    def __init__(self, in_size, reduction=4):
        super(SeModule, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_size, in_size // reduction, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(in_size // reduction),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_size // reduction, in_size, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(in_size),
            hsigmoid()
        )

    def forward(self, x):
        return x * self.se(x)


class Block(nn.Module):
    '''expand + depthwise + pointwise'''
    def __init__(self, kernel_size, in_size, expand_size, out_size, nolinear, semodule, stride):
        super(Block, self).__init__()
        self.stride = stride
        self.se = semodule

        self.conv1 = nn.Conv2d(in_size, expand_size, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(expand_size)
        self.nolinear1 = nolinear
        self.conv2 = nn.Conv2d(expand_size, expand_size, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, groups=expand_size, bias=False)
        self.bn2 = nn.BatchNorm2d(expand_size)
        self.nolinear2 = nolinear
        self.conv3 = nn.Conv2d(expand_size, out_size, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_size)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_size != out_size:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_size, out_size, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_size),
            )

    def forward(self, x):
        out = self.nolinear1(self.bn1(self.conv1(x)))
        out = self.nolinear2(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        if self.se != None:
            out = self.se(out)
        out = out + self.shortcut(x) if self.stride==1 else out
        return out


class MobileNetV3_Large(nn.Module):
    def __init__(self, num_classes=4):
        super(MobileNetV3_Large, self).__init__()
        self.conv1 = nn.Conv2d(4, 16, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.hs1 = hswish()

        self.bneck = nn.Sequential(
            Block(3, 16, 16, 16, nn.ReLU(inplace=True), None, 1),
            Block(3, 16, 64, 24, nn.ReLU(inplace=True), None, 2),
            Block(3, 24, 72, 24, nn.ReLU(inplace=True), None, 1),
            Block(5, 24, 72, 40, nn.ReLU(inplace=True), SeModule(40), 2),
            Block(5, 40, 120, 40, nn.ReLU(inplace=True), SeModule(40), 1),
            Block(5, 40, 120, 40, nn.ReLU(inplace=True), SeModule(40), 1),
            Block(3, 40, 240, 80, hswish(), None, 2),
            Block(3, 80, 200, 80, hswish(), None, 1),
            Block(3, 80, 184, 80, hswish(), None, 1),
            Block(3, 80, 184, 80, hswish(), None, 1),
            Block(3, 80, 480, 112, hswish(), SeModule(112), 1),
            Block(3, 112, 672, 112, hswish(), SeModule(112), 1),
            Block(5, 112, 672, 160, hswish(), SeModule(160), 1),
            Block(5, 160, 672, 160, hswish(), SeModule(160), 2),
            Block(5, 160, 960, 160, hswish(), SeModule(160), 1),
        )


        self.conv2 = nn.Conv2d(160, 960, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(960)
        self.hs2 = hswish()
        self.linear3 = nn.Linear(960, 1280)
        self.bn3 = nn.BatchNorm1d(1280)
        self.hs3 = hswish()
        self.linear4 = nn.Linear(1280, num_classes)
        self.init_params()
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

    def init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.hs3(self.bn3(self.linear3(out)))
        out = self.linear4(out)
        return out
    def extract(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 2)
        return out




class MobileNetV3_Small(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV3_Small, self).__init__()
        self.conv1 = nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.hs1 = hswish()

        self.bneck = nn.Sequential(
            Block(3, 16, 16, 16, nn.ReLU(inplace=True), SeModule(16), 2),
            Block(3, 16, 72, 24, nn.ReLU(inplace=True), None, 2),
            Block(3, 24, 88, 24, nn.ReLU(inplace=True), None, 1),
            Block(5, 24, 96, 40, hswish(), SeModule(40), 2),
            Block(5, 40, 240, 40, hswish(), SeModule(40), 1),
            Block(5, 40, 240, 40, hswish(), SeModule(40), 1),
            Block(5, 40, 120, 48, hswish(), SeModule(48), 1),
            Block(5, 48, 144, 48, hswish(), SeModule(48), 1),
            Block(5, 48, 288, 96, hswish(), SeModule(96), 2),
            Block(5, 96, 576, 96, hswish(), SeModule(96), 1),
            Block(5, 96, 576, 96, hswish(), SeModule(96), 1),
        )


        self.conv2 = nn.Conv2d(96, 576, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(576)
        self.hs2 = hswish()
        self.linear3 = nn.Linear(576, 1280)
        self.bn3 = nn.BatchNorm1d(1280)
        self.hs3 = hswish()
        self.linear4 = nn.Linear(1280, num_classes)
        self.init_params()

    def init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.hs3(self.bn3(self.linear3(out)))
        out = self.linear4(out)
        return out
        
    def extract(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 4)

        return out



def test():
    net = MobileNetV3_Small()
    x = torch.randn(2,3,224,224)
    y = net(x)
    print(y.size())

## Dataset

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [10]:
def get_annotation(id,input,anno_path='/content/labels'):
    anno = np.load(os.path.join(anno_path,'{:06d}.npy'.format(id)),allow_pickle=True).item()
    return anno.get(input)

In [11]:
import torch.utils.data as data
import os
import numpy as np
from torch.utils.data import Dataset
import cv2

class MiniDataset(Dataset):
    def __init__(self, base_p, label_f, depth, crops_rgb_f, label_name=['container capacity']):
      self.label_f = label_f #_f = folder
      self.depth = depth
      self.base = base_p
      self.label_name = label_name
      self.crops_rgb_f = crops_rgb_f
      self.samples = os.listdir(crops_rgb_f)
      self.ids = [ int(x.split('.')[0]) for x in self.samples]
      self.transform = transforms.Compose([
                                             transforms.Resize((320, 320)),
                                             transforms.ToTensor(),
                                             transforms.ConvertImageDtype(torch.float),
                                             ])
    def __len__(self):
      return len(self.ids)

    def __getitem__(self, idx):
      id_ = self.ids[idx]
        
      # depth
      depth = np.asarray(Image.open(os.path.join(self.depth,'{:06d}.png'.format(id_))))[:,:,np.newaxis]
      
      # rgb_cropped
      crop = np.asarray(Image.open(os.path.join(self.crops_rgb_f,'{:06d}.png'.format(id_))))

      h, w, c = crop.shape

      resX = 640 - h
      resY = 640 - w

      up = resX // 2
      down = up
      if resX % 2 != 0:
        down +=1

      left = resY // 2
      right = left

      if resY % 2 != 0:
        left += 1

      padding = transforms.Pad((left, up, right, down))

    
      image = Image.fromarray(np.concatenate((crop, depth), axis=2))
      image = padding(image)
      image = self.transform(image)
      # label
      label = np.array([get_annotation(id_,name,os.path.join(self.base, 'labels')) for name in self.label_name])

      return image, label

In [107]:
import copy
def computeScoreType1(gt, _est):
  gt = gt.cpu().data.numpy()
  _est = _est.cpu().data.numpy()
  est = copy.deepcopy(_est)
  assert (len(gt) == len(est))
  if all(x == -1 for x in est):
    return 0
  indicator_f = est > -1
  ec = np.exp(-(np.abs(gt - est) / gt)) * indicator_f
  score = np.sum(ec) / len(gt)
  return score

## Train

In [108]:
def myLoss(est, gt):
  ec = 1 - torch.exp(-(torch.abs(gt - est) / gt))
  score = torch.sum(ec) / len(gt)

  return score
  

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 2
train_split = 0.8
lr = 1e-4
epochs = 50
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb')
val_set = MiniDataset('/content/test/',
                        '/content/test/labels', 
                        '/content/test/crops_depth',
                        '/content/test/crops_rgb')
model = effnetv2_xl(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)



train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer, myLoss)
  loss_val, correct_val = evaluate(model, val_loader, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train, 100 * correct_train/num_train,
      loss_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "XL-my{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
0.7357405696268342
0.7358148734526937
0.7359936721591558


KeyboardInterrupt: ignored

In [111]:
def train(model, train_loader, optimizer, criterion = nn.L1Loss()):
  model.train()
  loss_train = 0.0
  correct_train = 0.0
  num_train = len(train_loader)
  for batch_idx, (audio, target) in enumerate(train_loader):
    audio = audio.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    outputs = model.forward(audio)
    loss = criterion(outputs, target)
    loss.backward()
    #nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    loss_train += loss.item() / (num_train * audio.shape[0] )

    correct_train += computeScoreType1(target, outputs) * audio.shape[0]

    
    
  
  return loss_train, correct_train


def evaluate(model, testloader, criterion = nn.L1Loss()):
  model.eval()
  loss_test = 0
  correct_test=0
  num_val = len(testloader)
  with torch.no_grad():
    for batch_idx, (audio, target) in enumerate(testloader):
      audio = audio.to(device)
      target = target.to(device)
      outputs = model.forward(audio)
      loss = criterion(outputs, target)
      loss_test += loss.item() / (num_val * audio.shape[0] )
      correct_test += computeScoreType1(target, outputs) * audio.shape[0]


  
  return loss_test, correct_test


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb')
val_set = MiniDataset('/content/test/',
                        '/content/test/labels', 
                        '/content/test/crops_depth',
                        '/content/test/crops_rgb')
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer, myLoss)
  loss_val, correct_val = evaluate(model, val_loader, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train, 100 * correct_train/num_train,
      loss_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.0199 train acc:24.11% val loss:0.0521 val acc:21.90%
2/100 train loss:0.0156 train acc:39.75% val loss:0.0206 val acc:44.07%


## Dimension

In [35]:
def train(model, train_loader, optimizer, criterion = nn.L1Loss()):
  model.train()
  loss_train = 0.0
  correct_train = 0.0
  num_train = len(train_loader)
  for batch_idx, (audio, target) in enumerate(train_loader):
    audio = audio.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    outputs = model.forward(audio)

    loss = criterion(outputs, target)
    loss.backward()
    #nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    loss_train += loss.item() / num_train

    for i in range(outputs.shape[1]):
      correct_train += computeScoreType1(target[:, i], outputs[:, i]) * \
      audio.shape[0] / audio.shape[1]
    
  
  return loss_train, correct_train


def evaluate(model, testloader, criterion = nn.L1Loss()):
  model.eval()
  loss_test = 0
  correct_test=0
  num_val = len(testloader)
  with torch.no_grad():
    for batch_idx, (audio, target) in enumerate(testloader):
      audio = audio.to(device)
      target = target.to(device)
      outputs = model.forward(audio)
      loss = criterion(outputs, target)
      loss_test += loss.item() / num_val   
      for i in range(outputs.shape[1]):
        correct_test += computeScoreType1(target[:, i], outputs[:, i]) * \
        audio.shape[0] /  audio.shape[1]

  
  return loss_test ,correct_test


In [36]:
print([os.path.join('/content/train/', 'labels') for name in ['width at the top', 'width at the bottom', 'height']])


['/content/train/labels', '/content/train/labels', '/content/train/labels']


In [37]:
def loss_func(est, gt):
  score = 0
  for i in range(est.shape[1]):
    ec = 1 - torch.exp(-(torch.abs(gt[:, i] - est[:, i]) / gt[:, i]))
    score += torch.sum(ec) / len(gt[:, i])
  
  score /= est.shape[1]
  
  return score

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 2
train_split = 0.8
lr = 1e-4
epochs = 200
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        ['width at the top', 'width at the bottom', 'height'])
val_set = MiniDataset('/content/test', 
                      '/content/test/labels',
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      ['width at the top', 'width at the bottom', 'height'])
model = effnetv2_xl().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)



train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer, loss_func)
  loss_val, correct_val = evaluate(model, val_loader, loss_func)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train, 100 * correct_train/num_train,
      loss_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "Dim{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/200 train loss:0.4439 train acc:35.71% val loss:0.2819 val acc:53.85%
2/200 train loss:0.1748 train acc:61.89% val loss:0.2176 val acc:58.65%
3/200 train loss:0.1584 train acc:63.12% val loss:0.2012 val acc:59.87%
4/200 train loss:0.1566 train acc:63.25% val loss:0.2433 val acc:56.87%
5/200 train loss:0.1513 train acc:63.65% val loss:0.1739 val acc:61.97%
6/200 train loss:0.1450 train acc:64.12% val loss:0.1613 val acc:62.85%
7/200 train loss:0.1377 train acc:64.68% val loss:0.1695 val acc:62.27%
8/200 train loss:0.1334 train acc:64.99% val loss:0.1617 val acc:62.89%


KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi